In [1]:
import torch
import gym 
from collections import defaultdict


env = gym.make("Taxi-v3", render_mode="human")
n_state = env.observation_space.n 
print("Number of states: ", n_state)
n_action = env.action_space.n 
print("Number of actions: ", n_action)
print("Reset state:", env.reset())
#env.render()

Number of states:  500
Number of actions:  6
Reset state: (133, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})


In [ ]:
def make_hashable(obj):
    """Recursively convert obj to a hashable type."""
    if isinstance(obj, dict):
        return tuple(sorted((k, make_hashable(v)) for k, v in obj.items()))
    elif isinstance(obj, (list, tuple)):
        return tuple(make_hashable(x) for x in obj)
    else:
        return obj

def gen_epsilon_greedy_policy(n_action, epsilon):
    def policy_fn(state, Q):
        probs = torch.ones(n_action)*epsilon / n_action
        hashed_state = make_hashable(state)
        best_action = torch.argmax(Q[hashed_state]).item()
        probs[best_action] += (1.0 - epsilon)
        action = torch.multinomial(probs, 1).item()
        return action
    return policy_fn

epsilon = 0.1
epsilon_greedy_policy = gen_epsilon_greedy_policy(n_action, epsilon)
   
def q_learning(env, gamma, n_episodes, alpha):
    n_action  = env.action_space.n
    Q = defaultdict(lambda: torch.zeros(n_action))
    for _ in range(n_episodes):
        state = env.reset()
        is_done = False
        while not is_done:
            hashed_state = make_hashable(state)
            print("this is the hashed state" , hashed_state)
            action = epsilon_greedy_policy(hashed_state, Q)  
            next_state, reward, is_done, info = env.step(action)
            hashed_next_state = make_hashable(next_state)
            print("this is the hashed state" , hashed)
            delta = reward + gamma * torch.max(Q[hashed_next_state]) - Q[hashed_state][action]
            Q[hashed_state][action] += alpha * delta
            length_episode[_] += 1
            total_reward_episode[_] += reward
            if is_done:
                break
            state = next_state
    policy = {}
    for state, action in Q.items():
        policy[state] = torch.argmax(action).item()
    return Q, policy 
n_episodes = 1000
length_episode = [0]*n_episodes
total_reward_episode = [0]*n_episodes

gamma = 1
alpha =0.1
optimal_Q, optimal_policy = q_learning(env, gamma, n_episodes, alpha)

TypeError: unhashable type: 'numpy.ndarray'